In [2]:
import os
from openai import AzureOpenAI

%run env_config_setting.ipynb

deploy_name = "gpt-35-turbo-16k"

api_key = openai.api_key
api_base = openai.api_base
api_version = openai.api_version


In [3]:
client = AzureOpenAI(
  azure_endpoint = api_base, #"https://azure-openai-test-s100.openai.azure.com/", 
  api_key = api_key, #"6cb5bfda3d7241dfb35c518668a4c099", #"s.getenv("AZURE_OPENAI_KEY"),  
  api_version = api_version #"2024-02-15-preview"
)

In [4]:

def chat_completion_request(messages, tools=None, tool_choice=None, model=deploy_name):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                },
                "required": ["location", "format"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get an N-day weather forecast",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "The temperature unit to use. Infer this from the users location.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    }
                },
                "required": ["location", "format", "num_days"]
            },
        }
    },
]

In [6]:
# without telling gpt information 
# so gpt response and ask user to provide more information

messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})

chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
print(assistant_message)

messages.append(assistant_message) # add to chat history

ChatCompletionMessage(content='Sure, can you please provide me with your current location?', role='assistant', function_call=None, tool_calls=None)


In [7]:
# tell gpt location
# so generate args for function [get_current_weather] (please take a look at the field: tool_calls)

messages.append({"role": "user", "content": "I'm in Taipei, Taiwan"})
chat_response = chat_completion_request(
    messages, tools=tools
)

assistant_message = chat_response.choices[0].message
print(assistant_message)


messages.append(assistant_message) # add messge part to chat history


ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_oasgP7MeCcfQgJCIo4cJ1wn9', function=Function(arguments='{\n  "location": "Taipei, Taiwan"\n}', name='get_current_weather'), type='function')])


In [8]:
import json

# define functions
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    print("enter get_current_weather!!")
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "24"})

def get_n_day_weather_forecast(location, unit, num_days):
    print("enter get_n_day_weather_forecast!!")
    return json.dumps({"location": location, "avg temperature": "72", "period": num_days, "unit": unit})


available_functions = {
    "get_current_weather": get_current_weather,
    "get_n_day_weather_forecast": get_n_day_weather_forecast,
}

In [13]:
# run tools here

def run_tools(assistant_message):
    tool_calls = assistant_message.tool_calls
    # check if the model wanted to call a function
    if tool_calls:
        # call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
    
        # send the info for each function call and function response to the model
        for tool_call in tool_calls:
            print("tool_call.function.name: " + tool_call.function.name + ", id:" + tool_call.id)
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = None
            if function_name == "get_current_weather":
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("unit"),
                )
            else:
                function_response = function_to_call(
                    location=function_args.get("location"),
                    unit=function_args.get("format"),
                    num_days=function_args.get("num_days"),
                )
            print("function_response: ====")
            print(function_response)
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  

            # extend conversation with function response
            second_response = client.chat.completions.create(
                model = deploy_name,
                messages = messages,
            )  # get a new response from the model where it can see the function response
                
            print("response for " + tool_call.function.name + " >>>>> ")
            resp = second_response.choices[0].message
            print(resp)
            messages.append(resp)

In [16]:
# tell gpt location and period
# so generate args for function [get_n_day_weather_forecas] (please take a look at the field: tool_calls)

messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Taipei, Taiwan over the next 3 days"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message) # add to chat history
print(assistant_message)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_r3eil0rHaCRrmf1P61C8iLem', function=Function(arguments='{\n  "location": "Taipei, Taiwan",\n  "num_days": 3\n}', name='get_n_day_weather_forecast'), type='function')])


In [18]:
run_tools(assistant_message)

tool_call.function.name: get_n_day_weather_forecast, id:call_r3eil0rHaCRrmf1P61C8iLem
enter get_n_day_weather_forecast!!
function_response: ====
{"location": "Taipei, Taiwan", "avg temperature": "72", "period": 3, "unit": null}
response for get_n_day_weather_forecast >>>>> 
ChatCompletionMessage(content='The weather in Taipei, Taiwan over the next 3 days is expected to have an average temperature of 72 degrees. However, I apologize for not providing more specific information such as the forecast for each day or details about precipitation. If you need additional details, please let me know.', role='assistant', function_call=None, tool_calls=None)


In [19]:
# force the model to use a specific function by using para tool_choice
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
                
choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice=choice
)
                
chat_response.choices[0].message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_F6cSkY3oVC8pZ8mFwXmkO9VK', function=Function(arguments='{\n  "location": "Toronto, Canada",\n  "num_days": 1\n}', name='get_n_day_weather_forecast'), type='function')])

In [ ]:
# how to run multiple tools and get response

In [20]:
#  force the model to not use a function at all by setting para tool_choice as "none"

messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
                
choice={"type": "function", "function": {"name": "get_n_day_weather_forecast"}}
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice="none"
)
                
chat_response.choices

[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, I can help you with that. Could you please specify if you'd like the current weather or a forecast for a specific number of days?", role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})]

In [17]:
#use pytho validate to check json data corresponding to JSON schema 

from jsonschema import validate

schema_example = {
    "type":"object",
    "properties":
    {
        "name":{"type": "string"},
        "age":{"type": "string"}
    },
    "required": ["name"],
}


validate(instance={"name":"John", "age":30}, schema=schema_example) # no error
validate(instance={"name":"John", "age":"30"}, schema=schema_example) # error: age type wrong
validate(instance={"age":30}, schema=schema_example) # error: name is required


ValidationError: 30 is not of type 'string'

Failed validating 'type' in schema['properties']['age']:
    {'type': 'string'}

On instance['age']:
    30